In [22]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Preprocess the Data

In [4]:
# Drop the non-beneficial ID columns 'EIN' and 'NAME'
df = df.drop(columns=['EIN', 'NAME'])

In [5]:
# Check the value counts of APPLICATION_TYPE and CLASSIFICATION columns for potential binning
print(df['APPLICATION_TYPE'].value_counts())
print(df['CLASSIFICATION'].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [6]:
# Choose a cutoff value and create lists of application types and classifications to be replaced
application_type_cutoff = 500
classification_cutoff = 1000

In [10]:
# Create lists of application types and classifications to be replaced
application_types_to_replace = df['APPLICATION_TYPE'].value_counts()[df['APPLICATION_TYPE'].value_counts() < application_type_cutoff].index.tolist()
classifications_to_replace = df['CLASSIFICATION'].value_counts()[df['CLASSIFICATION'].value_counts() < classification_cutoff].index.tolist()

In [11]:
# Replace less common application types and classifications with 'Other'
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].apply(lambda x: 'Other' if x in application_types_to_replace else x)
df['CLASSIFICATION'] = df['CLASSIFICATION'].apply(lambda x: 'Other' if x in classifications_to_replace else x)

In [18]:
# Encode categorical variables using one-hot encoding
df_encoded = pd.get_dummies(df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [19]:
# Split the data into features (X) and target (y)
X = df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = df_encoded['IS_SUCCESSFUL']

In [20]:
# Split the preprocessed data into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (27439, 43)
X_test shape: (6860, 43)
y_train shape: (27439,)
y_test shape: (6860,)


# Design and Optimize the Neural Network Model

In [24]:
# Define the model
model = Sequential()

# Define the number of input features
input_features = X_train.shape[1]  # Number of columns in X_train

# Input layer
model.add(Dense(units=128, activation='relu', input_dim=input_features))

# Additional hidden layers
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with custom hyperparameters
custom_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with adjusted hyperparameters
history = model.fit(X_train, y_train, epochs=100, batch_size=256, verbose=1, validation_split=0.2)


Epoch 1/100
86/86 [==============================] - 0s 1ms/step - loss: 26847.6543 - accuracy: 0.4752 - val_loss: 22544.5605 - val_accuracy: 0.5173
Epoch 2/100
86/86 [==============================] - 0s 978us/step - loss: 26052.1406 - accuracy: 0.5008 - val_loss: 25587.4023 - val_accuracy: 0.5171
Epoch 3/100
86/86 [==============================] - 0s 947us/step - loss: 20691.4805 - accuracy: 0.4780 - val_loss: 2640.4817 - val_accuracy: 0.4827
Epoch 4/100
86/86 [==============================] - 0s 953us/step - loss: 15134.1367 - accuracy: 0.4932 - val_loss: 11970.0918 - val_accuracy: 0.5173
Epoch 5/100
86/86 [==============================] - 0s 961us/step - loss: 3469.6379 - accuracy: 0.4933 - val_loss: 3473.3225 - val_accuracy: 0.4827
Epoch 6/100
86/86 [==============================] - 0s 960us/step - loss: 9968.1719 - accuracy: 0.4888 - val_loss: 869.9116 - val_accuracy: 0.4829
Epoch 7/100
86/86 [==============================] - 0s 972us/step - loss: 6073.7197 - accuracy: 0.494

Epoch 57/100
86/86 [==============================] - 0s 954us/step - loss: 0.6907 - accuracy: 0.5357 - val_loss: 0.6938 - val_accuracy: 0.5171
Epoch 58/100
86/86 [==============================] - 0s 946us/step - loss: 0.6907 - accuracy: 0.5357 - val_loss: 0.6936 - val_accuracy: 0.5171
Epoch 59/100
86/86 [==============================] - 0s 947us/step - loss: 0.6906 - accuracy: 0.5357 - val_loss: 0.6936 - val_accuracy: 0.5171
Epoch 60/100
86/86 [==============================] - 0s 942us/step - loss: 0.6906 - accuracy: 0.5357 - val_loss: 0.6935 - val_accuracy: 0.5171
Epoch 61/100
86/86 [==============================] - 0s 942us/step - loss: 0.6906 - accuracy: 0.5357 - val_loss: 0.6934 - val_accuracy: 0.5171
Epoch 62/100
86/86 [==============================] - 0s 939us/step - loss: 0.6906 - accuracy: 0.5357 - val_loss: 0.6934 - val_accuracy: 0.5171
Epoch 63/100
86/86 [==============================] - 0s 953us/step - loss: 0.6906 - accuracy: 0.5357 - val_loss: 0.6934 - val_accuracy:

In [25]:
# Check the structure of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               5632      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16001 (62.50 KB)
Trainable params: 16001 (62.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# Define the file path for saving the optimized model
optimized_model_file_path = 'AlphabetSoupCharity_Optimization.h5'

# Save the optimized model to an HDF5 file
model.save(optimized_model_file_path)

print(f"Optimized model saved to {optimized_model_file_path}")


Optimized model saved to AlphabetSoupCharity_Optimization.h5


/Users/ugochiprinceajuobi/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
